In [1]:
# Optimal pandas import
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

pd.options.display.max_colwidth = -1
pd.options.display.max_rows = 999

In [2]:
# Cargar info distritos.
## Necesitamos arreglar los nombres porque tenemor que mergear en base a eso.
## CNE DEBERIA USAR CODIGOS ESTANDARIZADOS DE PARTAMENTOS COMO CENSO INDEC.

In [3]:
from unidecode import unidecode

COD_PROV = pd.read_csv('./codigos/COD_PROVINCIAS.csv', ';', encoding = 'latin-1').rename(columns = {'NOMBRE': 'PROVINCIA'})
COD_DEP = pd.read_csv('./codigos/COD_DEPARTAMENTOS.csv', ';', encoding = 'latin-1').rename(columns = {'NOMBRE': 'DEPARTAMENTO'})
#insertar 0 en nombres de comuna, ej. Comuna 05
COD_DEP['dpto_merger_str'] = COD_DEP['DEPARTAMENTO'].str.replace('12 ', 'Doce'
                                            ).str.replace('1° ', 'Primero'
                                            ).str.replace('2 ', 'Dos '
                                            ).str.replace('25', 'Veinticinco'
                                            ).str.replace('9 ', 'Nueve ')

COD_DEP['dpto_merger_str'] = [unidecode(latin_str).lower() for latin_str in COD_DEP['DEPARTAMENTO']]
COD_DEP.loc[:8, 'dpto_merger_str'] = [s[:7] + '0' + s[7:] for s in COD_DEP.loc[:8]['dpto_merger_str'].values]

# Fix differences for BUENOS AIRES dptos
COD_DEP['dpto_merger_str'] = COD_DEP['dpto_merger_str'].str.replace('matanza', 'la matanza'
                                            ).str.replace('j. c. paz', 'jose c. paz'
                                            ).str.replace('roque saenz pena', 'presidente roque saenz pena '
                                            ).str.replace('coronel rosales', 'coronel de marina l. rosales'
                                            ).str.replace('general lamadrid', 'general la madrid'
                                            ).str.replace('las heras', 'general las heras'
                                            ).str.replace('general madariaga', 'general juan madariaga'
                                            ).str.replace('gonzales chaves', 'adolfo gonzales chaves'
                                            ).str.replace('f. ameghino', 'florentino ameghino'
                                            ).str.replace('juarez', 'benito juarez'
                                            ).str.replace('arrecifes', 'bartolome mitre'
                                            ).str.replace('lezama', 'chascomus')
geo_ref = pd.read_csv('./../../Dropbox/PBAshared/censo_geo/radio_ref.csv')
geo_ref['dpto_merger_str'] = geo_ref['NOMDPTO'].str.replace('12 ', 'Doce'
                                            ).str.replace('1° ', 'Primero'
                                            ).str.replace('2 ', 'Dos '
                                            ).str.replace('25', 'Veinticinco'
                                            ).str.replace('9 ', 'Nueve ')

geo_ref['dpto_merger_str'] = geo_ref['dpto_merger_str'].str.strip()
geo_ref['dpto_merger_str'] = [unidecode(latin_str).lower() for latin_str in geo_ref['dpto_merger_str']]
# geo_ref['dpto_merger_str'] = geo_ref['dpto_merger_str'].str.replace('arrecifes', 'bartolome mitre')

geo_ref.head()

# Las cantidades matchean. Bien!
# Tambien los codigos de provincia.
# geo_ref.groupby('PROV_REF_ID')['DPTO'].nunique()
# COD_DEP.groupby('CODIGO PROVINCIA')['DEPARTAMENTO'].nunique()

merged = COD_DEP.merge(geo_ref[['PROV_REF_ID','PROV','DPTO', 'dpto_merger_str']].drop_duplicates(),
              left_on = ['CODIGO PROVINCIA', 'dpto_merger_str'], right_on = ['PROV_REF_ID', 'dpto_merger_str'], how = 'left')
merged = merged.dropna() # perdemos dptos no matcheados del interior

IDcols = ['PROV_REF_ID', 'PROV', 'DPTO']
merged[IDcols] = merged[IDcols].applymap(int).applymap(str)


# Las provincias completas son CABA y GBA. Resto del pais hay como 30/350 dptos que no matchean
dpto_ref_BA = merged.loc[merged.PROV_REF_ID.isin(['1', '2'])].set_index('DEPARTAMENTO')[['DPTO', 'dpto_merger_str']]
display(dpto_ref_BA.sample(10))
# Ya esta todo linkeado con DPTOS geo, se pueden hacer mapas.

,DPTO,dpto_merger_str
DEPARTAMENTO,,
GENERAL ALVARADO,6280,general alvarado
LOMAS DE ZAMORA,6490,lomas de zamora
CORONEL DORREGO,6189,coronel dorrego
TANDIL,6791,tandil
VILLA GESELL,6868,villa gesell
CHIVILCOY,6224,chivilcoy
JUAREZ,6084,benito juarez
COMUNA 4,2004,comuna 04
MAGDALENA,6505,magdalena


# Data Electoral

## 2015

In [39]:
import unidecode
#Load data

primera_vuelta_2015 = pd.concat([pd.read_csv('./escrutinio/2015/FMESDN_0101.csv', ';'),
                            pd.read_csv('./escrutinio/2015/FMESDN_0202.csv', ';'),
                            pd.read_csv('./escrutinio/2015/FMESDN_0313.csv', ';'),
                            pd.read_csv('./escrutinio/2015/FMESPR_1424.csv', ';')])
primera_vuelta_2015['ELECCION'] = 'PV_2015'
# primera_vuelta_2015.sum()
# Nombres de listas
COD_PARTIDOS = pd.read_csv('./codigos/COD_PARTIDOS_2015.csv', ';', encoding = 'latin-1')[[u'Codigo Partido', u'Denominacion']]
COD_PARTIDOS.columns = [unidecode.unidecode(col) for col in COD_PARTIDOS.columns] # atender  Esta tilde
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.strip()
 # mergear
primera_vuelta_2015 = primera_vuelta_2015.merge(COD_PARTIDOS, left_on = u'CODIGO VOTOS', right_on = u'Codigo Partido')

# mergear parte geografica
primera_vuelta_2015 = primera_vuelta_2015.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'CODIGO PROVINCIA'])
# #fix
primera_vuelta_2015.columns = primera_vuelta_2015.columns.str.strip()
primera_vuelta_2015['CODIGO CIRCUITO'] = primera_vuelta_2015['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)
primera_vuelta_2015['CODIGO CIRCUITO'] = primera_vuelta_2015['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)

In [6]:
# primera_vuelta_2015 = primera_vuelta_2015.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'CODIGO PROVINCIA'])


## Paso 2017

In [40]:
import unidecode
import sys
# reload(sys)
# sys.setdefaultencoding("utf-8")

paso_2017 = pd.concat([pd.read_csv('./escrutinio/2017/paso2017-FMESDN_0101.csv', ';'),
                            pd.read_csv('./escrutinio/2017/paso2017-FMESDN_0202.csv', ';'),
                            pd.read_csv('./escrutinio/2017/paso2017-FMESDN_0313.csv', ';'),
                            pd.read_csv('./escrutinio/2017/paso2017-FMESDN_1424.csv', ';')])
paso_2017['ELECCION'] = 'paso_2017'

COD_PARTIDOS = pd.read_csv('./codigos/COD_PARTIDOS_2017_paso.csv', ';', encoding = 'latin-1')[[u'Código Partido', u'Denominacion']]
COD_PARTIDOS.columns = [unidecode.unidecode(col) for col in COD_PARTIDOS.columns]
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.strip()
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('CAMBIEMOS BUENOS AIRES', 'CAMBIEMOS')
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace('VAMOS JUNTOS', 'CAMBIEMOS')
COD_PARTIDOS['Denominacion'] = COD_PARTIDOS['Denominacion'].str.replace(u'UNIDAD PORTEÑA', 'UNIDAD CIUDADANA')

paso_2017 = paso_2017.merge(COD_PARTIDOS, left_on = u'CODIGO VOTOS', right_on = u'Codigo Partido')
paso_2017 = paso_2017.merge(COD_PROV).merge(COD_DEP, on = ['CODIGO DEPARTAMENTO', u'CODIGO PROVINCIA'])
# #fix
paso_2017.columns = paso_2017.columns.str.strip()
paso_2017['CODIGO CIRCUITO'] = paso_2017['CODIGO CIRCUITO'].astype(str).str.strip().str.zfill(4)

In [41]:
# Elegir:
# data_eleccion = paso_2017
data_eleccion = primera_vuelta_2015

In [42]:
data_eleccion.columns

Index(['CODIGO PROVINCIA', 'CODIGO DEPARTAMENTO', 'CODIGO CIRCUITO',
       'CODIGO MESA', 'AMBITO', 'CODIGO VOTOS', 'VOTOS', 'ELECCION',
       'Codigo Partido', 'Denominacion', 'PROVINCIA', 'DEPARTAMENTO',
       'dpto_merger_str'],
      dtype='object')

In [ ]:
## Un poquito de numerologia, porque dan muy redondo algunos numeros en la PASO 17 Buenos Aires.

# Cambiemos tiene 20.00% y UC 10.009986% de los votantes.

In [ ]:
(3310366 * 2) - 6555803

In [ ]:
3310366*0.000001

In [ ]:
6555803/64929

In [ ]:
3310366/0.10099 #32776326

In [ ]:
32776326 - 32775900.99009901

In [ ]:
3310366/32776326 #0.100999

In [ ]:
0.10099869033521329 - 0.101

In [ ]:
-1.30966478671668e-06 * 32776326

In [ ]:
6555803/32776326 #0.200016

In [ ]:
0.00001640818437064*32776326

In [43]:
# total_de_electores = 32776326 # Agosto 17
total_de_electores = 31754774 # Octubre 15

cuadro_nac = data_eleccion.groupby('Denominacion').sum().sort_values(by = 'VOTOS', ascending = False)/total_de_electores 
# SORPRENDENTE. Con paso 17
# Como puede ser que sean tan redondos los numeros?
# Estamos hablando de 6555803

In [35]:
ppales_listas

,CODIGO PROVINCIA,CODIGO DEPARTAMENTO,CODIGO CIRCUITO,CODIGO MESA,AMBITO,CODIGO VOTOS,VOTOS,ELECCION,Codigo Partido,Denominacion
0,1,1,0001,1,,9001,343,PV_2015,9001,ELECTORES
1,1,1,0001,2,,9001,346,PV_2015,9001,ELECTORES
2,1,1,0001,3,,9001,345,PV_2015,9001,ELECTORES
3,1,1,0001,4,,9001,344,PV_2015,9001,ELECTORES
4,1,1,0001,5,,9001,343,PV_2015,9001,ELECTORES
5,1,1,0001,6,,9001,344,PV_2015,9001,ELECTORES
6,1,1,0001,7,,9001,344,PV_2015,9001,ELECTORES
7,1,1,0001,8,,9001,344,PV_2015,9001,ELECTORES
8,1,1,0001,9,,9001,346,PV_2015,9001,ELECTORES
9,1,1,0001,10,,9001,344,PV_2015,9001,ELECTORES


In [44]:
# main = ['ELECTORES', 'VOTANTES', 'CAMBIEMOS', 'UNIDAD CIUDADANA', '1PAIS']
main = cuadro_nac.index[:9].values
ppales_listas = data_eleccion.loc[data_eleccion.Denominacion.isin(main)]

ppales_listas.groupby(['Denominacion','PROVINCIA']).sum()[['VOTOS']].sort_values(by = 'VOTOS', ascending = False).sort_index()/total_de_electores 


VOTOS
Denominacion                                      PROVINCIA                          
ALIANZA CAMBIEMOS                                 CHACO                      0.005458
                                                  CHUBUT                     0.001382
                                                  CIUDAD AUTONOMA DE BS.AS.  0.027889
                                                  CORDOBA                    0.033013
                                                  ENTRE RIOS                 0.008625
                                                  LA PAMPA                   0.002156
                                                  LA RIOJA                   0.002800
                                                  MENDOZA                    0.013598
                                                  MISIONES                   0.004563
                                                  NEUQUEN                    0.003256
                                                  RIO NEGRO                  0.002791
                                                  SALTA                      0.004538
                                                  SAN JUAN                   0.002646
                                                  SAN LUIS                   0.002641
                                                  SANTA CRUZ                 0.001369
                                                  SANTA FE                   0.022181
                                                  SANTIAGO DEL ESTERO        0.002565
                                                  TIERRA DEL FUEGO           0.000628
                                                  TUCUMAN                    0.007801
ALIANZA FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES BUENOS AIRES               0.012277
                                                  CIUDAD AUTONOMA DE BS.AS.  0.003348
                                                  CORDOBA                    0.003755
                                                  JUJUY                      0.000541
                                                  MENDOZA                    0.003881
                                                  MISIONES                   0.000182
                                                  NEUQUEN                    0.000629
                                                  RIO NEGRO                  0.000483
                                                  SALTA                      0.000604
                                                  SAN JUAN                   0.000187
                                                  SAN LUIS                   0.000152
                                                  SANTA CRUZ                 0.000169
                                                  SANTA FE                   0.001683
                                                  SANTIAGO DEL ESTERO        0.000254
                                                  TIERRA DEL FUEGO           0.000127
                                                  TUCUMAN                    0.000520
ALIANZA FRENTE PARA LA VICTORIA                   BUENOS AIRES               0.101179
                                                  CATAMARCA                  0.002610
                                                  CHACO                      0.010476
                                                  CHUBUT                     0.003090
                                                  CIUDAD AUTONOMA DE BS.AS.  0.013645
                                                  CORDOBA                    0.011959
                                                  CORRIENTES                 0.009102
                                                  ENTRE RIOS                 0.008875
                                                  FORMOSA                    0.005180
                                                  JUJUY                      0.003705
                                      

In [ ]:
0.269991 - .27

In [47]:
# -9.000000000036756e-06 * 32776322

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [48]:
ppales_listas = data_eleccion.loc[data_eleccion.Denominacion.isin(main)]
ppales_listas_BA = ppales_listas.loc[ppales_listas['PROVINCIA'].isin(['BUENOS AIRES'])]

cuadro = ppales_listas_BA.groupby(['Denominacion','CODIGO PROVINCIA', 'dpto_merger_str']).sum()[['VOTOS']].sort_values(by = 'VOTOS', ascending = False).sort_index().sort_values(by = 'VOTOS', ascending = False)
cuadro = cuadro.unstack([0, 1])

cuadro_pct = cuadro.copy()
cuadro_pct['VOTOS'] = cuadro['VOTOS']/cuadro['VOTOS']['ELECTORES']
cuadro_pct = cuadro_pct.add_suffix('_pct')
cuadro_cc = pd.concat([cuadro, cuadro_pct], axis = 1)

import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
s = cuadro_cc.style.background_gradient(cmap=cm)
s

In [ ]:
ppales_listas